In [ ]:
import torch
import torch.nn.functional as F
from sklearn.model_selection import train_test_split
import random

In [2]:
words = open('names.txt').read().splitlines()

In [3]:
char = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(char)}
itos = {i+1:s for i,s in enumerate(char)}
stoi['.'] = 0
itos[0] = '.'

In [4]:
block_size = 4 #using how many input letters it should predict the next letter.
x = []
y = []

for w in words:
    context = [0] * block_size #list storing the index of the char acc to the stoi/itos for storing in x.
    for ch in w + '.':
        idx = stoi[ch]
        x.append(context)
        y.append(idx)
        # print(''.join(itos[i] for i in context), '-->', itos[idx])
        context = context[1:] + [idx]

In [5]:
x = torch.tensor(x)
y = torch.tensor(y)

In [6]:
from sklearn.model_selection import train_test_split
xtr, xtemp, ytr, ytemp = train_test_split(x, y, test_size= 0.2, random_state= 42)

In [7]:
xval, xtest, yval, ytest = train_test_split(xtemp, ytemp, test_size= 0.5, random_state=43)

In [8]:
xtr.shape, ytr.shape, xval.shape, yval.shape, xtest.shape, ytest.shape

(torch.Size([182516, 4]),
 torch.Size([182516]),
 torch.Size([22815, 4]),
 torch.Size([22815]),
 torch.Size([22815, 4]),
 torch.Size([22815]))

In [9]:
g = torch.Generator().manual_seed(2147435354347)
C = torch.randn((27,10), generator=g)
w1 = torch.randn((40, 200), generator=g) * 0.2
b1 = torch.randn((200), generator=g) * 0.01
w2 = torch.randn((200,27), generator=g) * 0.01
b2 = torch.randn((27), generator=g) * 0
parameters = [C, w1, b1, w2, b2]

In [10]:
sum(p.numel() for p in parameters)

13897

In [11]:
for p in parameters:
    p.requires_grad = True

In [12]:
for i in range(200000):  

    #constructing minibatches
    ix = torch.randint(0, xtr.shape[0], (200,))


    #Forward Pass
    emb = C[xtr[ix]]
    hidden = torch.tanh(emb.view([-1,40]) @ w1 + b1)
    logits = hidden @ w2 + b2
    loss = F.cross_entropy(logits, ytr[ix])
    # print(loss.item())

    #Backward Pass
    for p in parameters:
        p.grad = None
    loss.backward()

    #Update
    for p in parameters:
        if i < 120000:
            p.data += -0.1 * p.grad
        else:
            p.data += -0.01 * p.grad

In [13]:
emb = C[xtr]
hidden = torch.tanh(emb.view([-1,40]) @ w1 + b1)
logits = hidden @ w2 + b2
loss = F.cross_entropy(logits, ytr)
print(loss.item())

1.9314604997634888


In [14]:
emb = C[xval]
hidden = torch.tanh(emb.view([-1,40]) @ w1 + b1)
logits = hidden @ w2 + b2
loss = F.cross_entropy(logits, yval)
print(loss.item())

2.0624115467071533


In [15]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 12)

for _ in range(5):
    
    out = []
    context = [0] * block_size
    while True:
      emb = C[torch.tensor([context])] # (1,block_size,d)
      h = torch.tanh(emb.view(1, -1) @ w1 + b1)
      logits = h @ w2 + b2
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

kahde.
zakaridu.
dhay.
quay.
ro.
